In [1]:
import numpy as np
import torch
import torchvision
from torch import nn
from __future__ import print_function, division
import matplotlib.patches as patches
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader,random_split
from torchvision import transforms, utils
import cv2
from PIL import ImageDraw, Image
from models import *
from utils.utils import *
#from utils.datasets import *
from utils.parse_config import *

In [2]:
class LiverDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, ano_dir, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.anot = pd.read_csv(csv_file)
        self.ano_dir=ano_dir
        self.root_dir = root_dir
        self.transform = transform
        
        self.imgs = list(sorted(os.listdir(root_dir)))
        self.masks = list(sorted(os.listdir(ano_dir)))

    def __len__(self):
        return len(self.anot)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

    
         # images-done   
        img_name = os.path.join(self.root_dir,
                                self.anot.iloc[idx, 0])
        name=self.anot.iloc[idx, 0]
        image= np.array(Image.open(img_name), dtype=np.uint8)

        
        if len(image.shape)==2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
          
        #annotations
        mask_path = os.path.join(self.ano_dir, self.anot.iloc[idx, 0])
        
        bbox = []
        labels=[]
        im=cv2.imread(mask_path)
        im2=cv2.imread(mask_path,0)
        #im2 = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        et,thresh = cv2.threshold(im2,127,255,0)
        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        if len(contours)!=0:
            for k in range(0, len(contours)):
                cnt = contours[k]
                mask = np.zeros(im2.shape,np.uint8)
                #cv2.drawContours(mask,[cnt],0,255,-1)
                xmin,ymin,w,h = cv2.boundingRect(cnt)
                #cv2.rectangle(im,(x,y),(x+w,y+h),(0,255,0),2)
                #xmax=xmin+w
                #ymax=ymin+h
                bbox.append([0,xmin, ymin, w, h])
                labels.append(1)
        bbox = np.array(bbox,dtype=np.float32)
        
        #resizing
        
        image = transform.resize(image, (416, 416))
        image= np.array(image, dtype=np.uint8)

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        #bbox[:,1:5] = bbox[:,1:5] * [416 / w]
        
        
        
        
        
 
        sample= {'bbox': bbox,'labels':labels,"name":name} 

        if self.transform:
                         image = self.transform(image)
        
            

        return image, sample

In [3]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self,image):
        image= image

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        #image= np.expand_dims(image, axis=2)
        image= np.array(image)
        image = image.transpose((2,0,1))
        image=torch.from_numpy(image)
        #bbox=torch.from_numpy(bbox)
        #labels=torch.from_numpy(labels)
        #image = image[None, :, :,:]
        image=image.float()
        
        return image

In [4]:
jitter=transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0, hue=0)
data_transform = transforms.Compose([transforms.ToPILImage(),transforms.RandomApply([jitter], p=0.1),ToTensor()])
transformed_dataset =LiverDataset(csv_file="C:/Users/Zuzana/Desktop/faster_rcnn/building/train_names.csv",ano_dir="C:/Users/Zuzana/Desktop/skola/bakalarka/ground_final",
                                  root_dir="C:/Users/Zuzana/Desktop/faster_rcnn/building/images",transform=data_transform)

In [5]:
trainloader = DataLoader(transformed_dataset, batch_size=1, num_workers=0, shuffle=False, drop_last=True)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}..")

Running on cpu..


In [7]:
model_config_path="config/yolov3.cfg"
#data_config_path="config/coco.data"
weights_path="config/yolov3.weights"
#use_cuda=True
#image_folder = "data/new_images1"
batch_size = 4
epochs=20
checkpoint_interval=1
conf_thres = 0.6
nms_thres = 0.4
checkpoint_dir="checkpoints"
class_path ="config/coco.names"
img_size=416
n_cpu=0
os.makedirs("checkpoints", exist_ok=True)



In [8]:
# Get hyper parameters
hyperparams = parse_model_config(model_config_path)[0]
learning_rate = float(hyperparams["learning_rate"])
momentum = float(hyperparams["momentum"])
decay = float(hyperparams["decay"])
burn_in = int(hyperparams["burn_in"])

In [9]:
#data_config = parse_data_config(data_config_path)
# Initiate model
model = Darknet(model_config_path)
model.load_weights(weights_path)
model=model.to(device=device)
#Tensor = torch.cuda.FloatTensor if device else torch.FloatTensor

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

C:\Users\Zuzana\venv\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [10]:
import warnings


In [11]:
for epoch in range(10):
    #running_loss=0
    for i,[image,sample] in enumerate(trainloader):
        image=image.to(device=device)
        bbox=sample["bbox"].to(device=device)
        labels=sample["labels"]
        #targets = [{'boxes':torch.FloatTensor(sample['bbox'][0,:,:]),'labels':torch.LongTensor(sample['labels'])}]
        model.train()
        optimizer.zero_grad()
        warnings.filterwarnings("ignore", category=FutureWarning)
        loss = model(image, bbox)
        loss.backward()
        optimizer.step()
        
        #model.train()
        #loss = model(image, targets)
    
        #loss_sum = sum(lss for lss in loss.values())
    
        #optimizer.zero_grad()
        #loss_sum.backward()
        #optimizer.step()
        #running_loss += loss_sum
        print(f"Epoch number {epoch},Iteration nummber {i}, LOSS: {loss}")
        
    #loss = running_loss/len(transformed_dataset)
    #training_loss.append(loss)
    #torch.save(model.state_dict(), os.path.join(models_path, "faster_rcnn1.pt")) #ukladanie modelu po každej epoche



C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.

Epoch number 0,Iteration nummber 0, LOSS: 78.57954406738281


C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.

Epoch number 0,Iteration nummber 1, LOSS: 15.439188957214355


C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead.
C:\w\1\s\windows\pytorch\aten\src\ATen/native/IndexingUtils.h:20: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.

Epoch number 0,Iteration nummber 2, LOSS: 5.863829612731934


IndexError: index 14 is out of bounds for dimension 2 with size 13